<a href="https://colab.research.google.com/github/prinsherbert/girrgorr/blob/master/example%20notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython

import numpy
from datetime import datetime, timedelta
from matplotlib import pyplot
from tqdm.auto import tqdm

import pandas
import rpy2.robjects as robjects
from IPython.display import display

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


# Create a synthetic time series

of three days, with some specific nights where the sensor has no movement as defined by the intervals in `nights`.

In [ ]:
t0 = datetime(2020, 7, 24, 12, 0, 0)
dt = timedelta(microseconds=10000)
n = 100*60*60*24*3

t_ = numpy.linspace(0, 3*24*60*60, n)
t = numpy.arange(t0, t0+n*dt, dt)

In [ ]:
nights = [
    (datetime(2020, 7, 24, 21, 32, 12, 230), datetime(2020, 7, 25, 6, 43, 23, 650)),
    (datetime(2020, 7, 26, 2, 12, 45, 450), datetime(2020, 7, 26, 11, 56, 45, 890)),
    (datetime(2020, 7, 26, 22, 56, 23, 320), datetime(2020, 7, 27, 7, 34, 35, 760))
]

nighttime = numpy.zeros(n, numpy.bool)
for a, b in tqdm(nights):
    nighttime |= (t >= a) & (t <= b)
    
daytime = 1 - nighttime

In [ ]:
x = daytime * numpy.sin(0.5 * 2 * numpy.pi * t_) * (0.5 + 0.5 * numpy.sin(0.001 * numpy.pi * t_) ** 2)
y = daytime * numpy.sin(2 * 2 * numpy.pi * t_) * (0.5 + 0.5 * numpy.sin(0.003 * numpy.pi * t_) ** 2)
z = daytime * numpy.sin(4 * 2 * numpy.pi * t_) * (0.5 + 0.5 * numpy.sin(0.0005 * numpy.pi * t_) ** 2)

# Write as actigraph CSV files

In [ ]:
def line(t, x, y, z):
    return t.strftime("%d-%m-%Y %H:%M:%S") + f".{t.microsecond // 1000:03d},{x:5.03f},{y:5.03f},{z:5.03f}"

In [ ]:
!mkdir -p data

with open('data/test.csv', 'w') as f:
    f.write(f"""------------ Data File Created By ActiGraph GT3X+ ActiLife v6.13.3 Firmware v1.7.1 date format d-M-yyyy at 100 Hz  Filter Normal -----------
Serial Number: TAS1F12170002
Start Time {t0.strftime('%H:%M:%S')}
Start Date {t0.strftime('%d-%m-%Y')}
Epoch Period (hh:mm:ss) 00:00:00
Download Time 16:20:37
Download Date 28-07-2020
Current Memory Address: 0
Current Battery Voltage: 3.96     Mode = 12
--------------------------------------------------
Timestamp,Accelerometer X,Accelerometer Y,Accelerometer Z
""")
    for i in tqdm(range(len(t))):
        f.write(line(t0 + i * dt, x[i], y[i], z[i]) + '\n')

# Install and run GGIR

In [ ]:
%%R

install.packages("GGIR")

In [ ]:
!rm -rf "/content/data-out/"

In [ ]:
with open('process.R', 'w') as f:
  f.write("""

rm(list=ls())
graphics.off()
library(GGIR)

mode = c(0, 1, 2)
dayborder = 4

datadir = "/content/data/"
outputdir = "/content/data-out/"

stt <- Sys.time()
print(paste0("Starting at: ", stt))

g.shell.GGIR(#=======================================
             mode=mode, #specify above
             datadir=datadir, #specify above
             outputdir=outputdir, #specify above
             f0=c(), #specify above
             f1=c(), #specify above
             overwrite = FALSE, #overwrite previous milestone data?
             do.imp=FALSE, # Do imputation? (recommended)
             idloc=1, #id location (1 = file header, 2 = filename)
             print.filename=TRUE,
             storefolderstructure = FALSE,
             

             windowsizes = c(5,900,3600), #Epoch length, non-wear detection resolution, non-wear detection evaluation window
             do.cal=TRUE, # Apply autocalibration? (recommended)
             do.enmo = TRUE, #Needed for physical activity analysis
             do.en=TRUE,
             do.anglez=TRUE, #Needed for sleep detection
             do.angley=TRUE,
             do.anglex=TRUE,
             do.roll_med_acc_x=TRUE,
             do.roll_med_acc_y=TRUE,
             do.roll_med_acc_z=TRUE,
             do.dev_roll_med_acc_x=TRUE,
             do.dev_roll_med_acc_y=TRUE,
             do.dev_roll_med_acc_z=TRUE,
             chunksize=1.0, #size of data chunks to be read (value = 1 is maximum)
             desiredtz = "Europe/London",
             printsummary=TRUE,
             minloadcrit=46,
             epochvalues2csv = FALSE,
             
             
             
             strategy = 1, #Strategy (see tutorial for explanation)
             ndayswindow=7, #only relevant when strategy = 3
             hrs.del.start = 0, # Only relevant when strategy = 2. How many HOURS need to be ignored at the START of the measurement?
             hrs.del.end = 0, # Only relevant when strategy = 2. How many HOURS need to be ignored at the END of the measurement?
             maxdur = 8, # How many DAYS of measurement do you maximumally expect?
             includedaycrit = 0, # number of minimum valid hours in a day to attempt physical activity analysis
             L5M5window = c(0,24), #window over which to calculate L5 and M5
             M5L5res = 10, #resolution in minutes of M5 and L5 calculation
             winhr = 5, # size of M5 and L5 (5 hours by default)
             qlevels = c(), #c(c(1380/1440),c(1410/1440)), #quantiles to calculate, set value at c() if you do not want quantiles
             qwindow=c(0,24), #window over which to calculate quantiles
             ilevels = c(),#c(seq(0,400,by=50),8000), #acceleration values (metric ENMO) from which a frequency distribution needs to be derived, set value at c() if you do not want quantiles
             mvpathreshold = c(100), #MVPA (moderate and vigorous physical activity threshold
             window.summary.size = 10,
             dayborder = dayborder, # dayborder is the hour at which one day becomes the next day
             bout.metric = 4,
             closedbout=FALSE,
             #-----------------------------------
             # Report generation
             #-------------------------------
             # Key functions: Generating reports based on meta-data
             do.report=c(2), #for what parts does and report need to be generated?)
             visualreport = FALSE) 

fnsh <- Sys.time()

print(paste0("DONE. At: ", fnsh))
print(paste0("Took ", difftime(fnsh, stt, units = "mins"), " minutes"))
""")

!mkdir -p /content/data-out/
!Rscript process.R

[1] "Starting at: 2020-07-27 11:05:48"

   Do not forget to cite GGIR in your publications via a version number and
   Migueles et al. 2019 JMPB. doi: 10.1123/jmpb.2018-0063. 
   See also: https://cran.r-project.org/package=GGIR/vignettes/GGIR.html#citing-ggir 

________________________________________________________________________________
Part 1

Checking that user has read access permission for all files in data directory: Yes
Checking that user has write access permission for directory specified by argument outputdir: Yes

parallel processing not possible because number of available cores (2) < 4
File name: test.csv
P1 file 1

Investigate calibration of the sensors with function g.calibrate:

Loading chunk: 1 No non-movement found
 2 No non-movement found
 3 No non-movement found
 4 No non-movement found
 5 No non-movement found
 6 No non-movement found
 7 No non-movement found

Summary of autocalibration procedure:

Status: recalibration not done because no non-movement data avai

# Show output of part 2

The 'ENMO' of part 2 is plot per day seperately. This shows that the nigth/sleep-time is considered the same here, however they are different in the original signal.

Event though the original signal has varying bedtime and wake-up moments, the result of step 2 is only 'silent' between 02:11:40 and 06:42:15, all other times
have a signal, even when the original was silent. In the plot this behavior is visualized as the difference between the red and blue line.

These are the approximate night times for the original signal.

| day        | to sleep | wake up |
|------------|----------|---------|
| 2020-07-24 | 21:32    |  6:44   |
| 2020-07-25 | 12:46    | 11:57   |
| 2020-07-26 | 22:56    |  7:35   |


In [ ]:
%%R

data <- load('data-out/output_data/meta/ms2.out/test.csv.RData')
timeseries = IMP$metashort

timeseries

                    timestamp         ENMO        EN anglex angley anglez
1    2020-07-24T12:00:00+0100 0.0000000000 0.6669347    NaN    NaN    NaN
2    2020-07-24T12:00:05+0100 0.0000000000 0.6781892    NaN    NaN    NaN
3    2020-07-24T12:00:10+0100 0.0000000000 0.6899392    NaN    NaN    NaN
4    2020-07-24T12:00:15+0100 0.0000000000 0.7021016    NaN    NaN    NaN
5    2020-07-24T12:00:20+0100 0.0000000000 0.7146379    NaN    NaN    NaN
6    2020-07-24T12:00:25+0100 0.0000000000 0.7273466    NaN    NaN    NaN
7    2020-07-24T12:00:30+0100 0.0000000000 0.7401931    NaN    NaN    NaN
8    2020-07-24T12:00:35+0100 0.0000000000 0.7530071    NaN    NaN    NaN
9    2020-07-24T12:00:40+0100 0.0002007540 0.7657533    NaN    NaN    NaN
10   2020-07-24T12:00:45+0100 0.0016742008 0.7782582    NaN    NaN    NaN
11   2020-07-24T12:00:50+0100 0.0050202058 0.7904521    NaN    NaN    NaN
12   2020-07-24T12:00:55+0100 0.0087496092 0.8022133    NaN    NaN    NaN
13   2020-07-24T12:01:00+0100 0.012334

From cffi callback <function _consolewrite_ex at 0x7f91e01ab158>:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/rpy2/rinterface_lib/callbacks.py", line 131, in _consolewrite_ex
    consolewrite_print(s)
  File "/usr/local/lib/python3.6/dist-packages/rpy2/rinterface_lib/callbacks.py", line 114, in consolewrite_print
    print(s, end='', flush=True)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py", line 349, in flush
    if not evt.wait(self.flush_timeout):
  File "/usr/lib/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/python3.6/threading.py", line 299, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt


Streaming output truncated to the last 5000 lines.
601        0.015576        0.0e+00       0.003000           0.252662
602       -0.015632        0.0e+00       0.003000           0.254172
603        0.015760        0.0e+00       0.003000           0.255804
604       -0.015806        0.0e+00       0.003000           0.257548
605        0.015918        0.0e+00       0.003000           0.259366
606       -0.016066        0.0e+00       0.003000           0.261310
607        0.016188        0.0e+00       0.003000           0.263384
608       -0.016330        0.0e+00       0.003000           0.265530
609        0.016514        0.0e+00       0.003000           0.267748
610       -0.016612        0.0e+00       0.003000           0.270072
611        0.016792        0.0e+00       0.003000           0.272570
612       -0.016936        0.0e+00       0.003000           0.275028
613        0.017078        0.0e+00       0.003000           0.277648
614       -0.017280        0.0e+00       0.003000   

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



           0.561896           0.452944
6699           0.550582           0.450508
6700           0.537648           0.448048
6701           0.524368           0.445576
6702           0.511024           0.443150
6703           0.496508           0.440716
6704           0.482290           0.438268
6705           0.467750           0.435874
6706           0.452802           0.433462
6707           0.438872           0.431076
6708           0.424278           0.428684
6709           0.410250           0.426314
6710           0.397284           0.423946
6711           0.384162           0.421616
6712           0.371816           0.419208
6713           0.361230           0.416948
6714           0.350632           0.414656
6715           0.341102           0.412332
6716           0.332674           0.410080
6717           0.326014           0.407814
6718           0.320484           0.405570
6719           0.315862           0.403346
6720           0.312584           0.401166
6721           

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
timeseries

In [ ]:
df = pandas.DataFrame({
    col: numpy.array(timeseries.rx2(col))
    for col in timeseries.colnames
})

df['day'] = df['timestamp'].str[:10]

In [ ]:
day_labels = {}
done = 0
for i, (day, rows) in enumerate(df.groupby('day')):
    n_ = len(rows)
    offset = sum(a * b for a, b in zip(map(int, rows['timestamp'].values[0][11:-5].split(':')), [3600, 60, 1])) // 5
    tt = numpy.arange(offset, offset + len(rows))
    pyplot.plot(tt, 4*i + (rows['ENMO'].values > 0), '-b')
    
    acc = x[::500][done:done+n_]**2 + y[::500][done:done+n_] **2 + z[::500][done:done+n_] ** 2
    tt = numpy.arange(offset, (offset + len(rows)))
    pyplot.plot(tt, (4*i + 2) + (acc>0), '-r')
    done += n_
    
    day_labels[4*i] = day + ' step 2'
    day_labels[4*i+2] = ' raw    '
    
pyplot.yticks(list(day_labels.keys()), list(day_labels.values()))

pyplot.xticks(list(range(0, 24*60*12+1, 60*12)), list(range(24)) + [0])

pyplot.xlabel('time of the day')

pyplot.ylabel('acceleration of raw signal\nand ENMO after two steps in GGIR')

pyplot.legend(['ENMO output step 2', 'orinal signal'])

None